> Please run this in python310 and with a spacy kernel.

## Basic Usage

In [1]:
from termcolor import colored
import time
from fastcoref import spacy_component
from fastcoref import FCoref
import spacy

C:\Users\tyler\projects\Atlas\venv\spacy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sm_nlp = spacy.load("en_core_web_sm", exclude=["parser", "lemmatizer", "ner", "textcat"])
sm_nlp.add_pipe("fastcoref")
lg_nlp = spacy.load("en_core_web_lg", exclude=["parser", "lemmatizer", "ner", "textcat"])
lg_nlp.add_pipe("fastcoref")
ling_nlp = spacy.load("en_core_web_sm", exclude=["parser", "lemmatizer", "ner", "textcat"])
ling_nlp.add_pipe(
   "fastcoref", 
   config={'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref', 'device': 'cpu'}
)

01/08/2023 19:59:07 - INFO - 	 missing_keys: []
01/08/2023 19:59:07 - INFO - 	 unexpected_keys: []
01/08/2023 19:59:07 - INFO - 	 mismatched_keys: []
01/08/2023 19:59:07 - INFO - 	 error_msgs: []
01/08/2023 19:59:07 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
01/08/2023 19:59:09 - INFO - 	 missing_keys: []
01/08/2023 19:59:09 - INFO - 	 unexpected_keys: []
01/08/2023 19:59:09 - INFO - 	 mismatched_keys: []
01/08/2023 19:59:09 - INFO - 	 error_msgs: []
01/08/2023 19:59:09 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
01/08/2023 19:59:15 - INFO - 	 missing_keys: []
01/08/2023 19:59:15 - INFO - 	 unexpected_keys: []
01/08/2023 19:59:15 - INFO - 	 mismatched_keys: []
01/08/2023 19:59:15 - INFO - 	 error_msgs: []
01/08/2023 19:59:15 - INFO - 	 Model Parameters: 590.0M, Transformer: 434.6M, Coref head: 155.4M


In [3]:
text = 'Alice goes down the rabbit hole. Where she would discover a new reality beyond her expectations.'
doc = sm_nlp(text)
doc._.coref_clusters

01/08/2023 19:59:15 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.00ba/s]
01/08/2023 19:59:16 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.99it/s]


[[(0, 5), (39, 42), (79, 82)]]

## Utility Functions

In [4]:
COLORS = ['green', 'red', 'yellow', 'blue', 'magenta', 'cyan']

def get_color(index):
    return COLORS[index % len(COLORS)]

def cprint(text, color, newline=True):
    print(colored(text, color), end='\n' if newline else ' ')
    
def render_clusters(prediction, text):
    corefs = prediction._.coref_clusters
    for i, cluster in enumerate(corefs):
        cprint(f'Cluster {i}', get_color(i))
        for phrase_start, phrase_end in cluster:
            cprint(f'{text[phrase_start:phrase_end + 1]} - [{phrase_start}, {phrase_end}]', get_color(i))
            
def predict_and_time(document, pipeline):
    start = time.time()
    prediction = pipeline(document)
    end = time.time()
    print(document)
    render_clusters(prediction, document)
    print(f"Elapsed Time: {(end - start) * 1000}ms");

## Excerpt Tests (`sm_nlp`)

In [5]:
short_excerpt = 'In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. ' \
'It can also designate a nation or any inhabited place that has no system of government or central rule. ' \
'Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions. ' \
'These institutions or free associations are generally modeled on nature.'

In [6]:
predict_and_time(short_excerpt, sm_nlp)

01/08/2023 19:59:16 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 63.99ba/s]
01/08/2023 19:59:16 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.26it/s]

In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions. These institutions or free associations are generally modeled on nature.
Cluster 0
anarchy  - [20, 27]
It  - [123, 125]
Cluster 1
voluntary institutions. - [321, 343]
These institutions or free associations  - [345, 384]
Elapsed Time: 344.4647789001465ms


In [7]:
longer_article = 'Anarchy is a society without a government. It may also refer to a society or group of people that entirely rejects a set hierarchy. '\
'Anarchy was first used in English in 1539, meaning "an absence of government". Pierre-Joseph Proudhon adopted anarchy and anarchist in his 1840 treatise What Is Property? '\
'to refer to anarchism, a new political philosophy and social movement that advocates stateless societies based on free and voluntary associations. '\
'Anarchists seek a system based on the abolition of all coercive hierarchy, in particular the state, and many advocate for the creation of a system of direct democracy, '\
'worker cooperatives or privatization. In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. '\
'It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists '\
'who propose replacing government with voluntary institutions. These institutions or free associations are generally modeled on nature since they can represent '\
'concepts such as community and economic self-reliance, interdependence, or individualism. Although anarchy is often negatively used as a synonym of chaos or '\
'societal collapse or anomie, this is not the meaning that anarchists attribute to anarchy, a society without hierarchies. '\
'Proudhon wrote that anarchy is "Not the Daughter But the Mother of Order."'

In [8]:
predict_and_time(longer_article, sm_nlp)

01/08/2023 19:59:16 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 63.99ba/s]
01/08/2023 19:59:16 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.45it/s]

Anarchy is a society without a government. It may also refer to a society or group of people that entirely rejects a set hierarchy. Anarchy was first used in English in 1539, meaning "an absence of government". Pierre-Joseph Proudhon adopted anarchy and anarchist in his 1840 treatise What Is Property? to refer to anarchism, a new political philosophy and social movement that advocates stateless societies based on free and voluntary associations. Anarchists seek a system based on the abolition of all coercive hierarchy, in particular the state, and many advocate for the creation of a system of direct democracy, worker cooperatives or privatization. In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions.

In [9]:
very_long_article = 'Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to '\
'local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. '\
'The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). '\
'While Somalia\'s formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and '\
'administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. '\
'In the case of the Transitional National Government and its successor the Transitional Federal Government, '\
'new interim judicial structures were formed through various international conferences. Despite some significant political differences between '\
'them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of previous '\
'Somali administrations. These similarities in civil law included a charter which affirms the primacy of Muslim shari\'a or religious law, '\
'although in practice shari\'a is applied mainly to matters such as marriage, divorce, inheritance and civil issues. '\
'The charter assured the independence of the judiciary which in turn was protected by a judicial committee; a three-tier judicial system '\
'including a supreme court, a court of appeals and courts of first instance (either divided between district and regional courts, or a single court per region); '\
'and the laws of the civilian government which were in effect prior to the military coup d\'état that saw the Barre regime into power remain '\
'in forced until the laws are amended. '\
'Economist Alex Tabarrok claimed that Somalia in its stateless period provided a "unique test of the theory of anarchy", in some aspects near of '\
'that espoused by anarcho-capitalists such as David D. Friedman and Murray Rothbard. Nonetheless, both anarchists and some anarcho-capitalists '\
'such as Walter Block argue that Somalia was not an anarchist society.'

In [10]:
predict_and_time(very_long_article, sm_nlp)

01/08/2023 19:59:17 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 62.47ba/s]
01/08/2023 19:59:17 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.20it/s]

Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). While Somalia's formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. In the case of the Transitional National Government and its successor the Transitional Federal Government, new interim judicial structures were formed through various international conferences. Despite some significant political differences between them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of pr

In [11]:
angelfish_article = "The queen angelfish (Holacanthus ciliaris), also known as the blue angelfish, "\
"golden angelfish, or yellow angelfish, is a species of marine angelfish found in the western Atlantic Ocean. "\
"It is a benthic (ocean floor) warm-water species that lives in coral reefs. "\
"It is recognized by its blue and yellow coloration and a distinctive spot or 'crown' on its forehead. "\
"This crown distinguishes it from the closely related and similar-looking Bermuda "\
"blue angelfish (Holacanthus bermudensis), with which it overlaps in range and can interbreed. "\
"Adult queen angelfish are selective feeders and primarily eat sponges. Their social structure consists "\
"of harems which include one male and up to four females. "\
"They live within a territory where the females forage separately and are tended to by the male. "\
"Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. "\
"Juveniles of the species have different coloration than adults and act as cleaner fish."\
"Adult queen angelfish are selective feeders and primarily eat sponges. "\
"Their social structure consists of harems which include one male and up to four females. "\
"They live within a territory where the females forage separately and are tended to by the male. "\
"Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. "\
"Juveniles of the species have different coloration than adults and act as cleaner fish. "\
"The queen angelfish is popular in the aquarium trade and has been a particularly common exported "\
"species from Brazil. In 2010, the queen angelfish was assessed as least concern by the International "\
"Union for Conservation of Nature as the wild population appeared to be stable." 

In [12]:
predict_and_time(angelfish_article, sm_nlp)

01/08/2023 19:59:17 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 63.99ba/s]
01/08/2023 19:59:17 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.18it/s]

The queen angelfish (Holacanthus ciliaris), also known as the blue angelfish, golden angelfish, or yellow angelfish, is a species of marine angelfish found in the western Atlantic Ocean. It is a benthic (ocean floor) warm-water species that lives in coral reefs. It is recognized by its blue and yellow coloration and a distinctive spot or 'crown' on its forehead. This crown distinguishes it from the closely related and similar-looking Bermuda blue angelfish (Holacanthus bermudensis), with which it overlaps in range and can interbreed. Adult queen angelfish are selective feeders and primarily eat sponges. Their social structure consists of harems which include one male and up to four females. They live within a territory where the females forage separately and are tended to by the male. Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. Juveniles of the species have different coloration than adults and act as cleaner fish.
Cluster 0

## Excerpt Tests (`lg_nlp`)

In [13]:
predict_and_time(short_excerpt, lg_nlp)

01/08/2023 19:59:18 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.01ba/s]
01/08/2023 19:59:20 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.20it/s]

In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions. These institutions or free associations are generally modeled on nature.
Cluster 0
anarchy  - [20, 27]
It  - [123, 125]
Cluster 1
voluntary institutions. - [321, 343]
These institutions or free associations  - [345, 384]
Elapsed Time: 2310.854196548462ms


In [14]:
predict_and_time(longer_article, lg_nlp)

01/08/2023 19:59:20 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 60.14ba/s]
01/08/2023 19:59:22 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.90it/s]

Anarchy is a society without a government. It may also refer to a society or group of people that entirely rejects a set hierarchy. Anarchy was first used in English in 1539, meaning "an absence of government". Pierre-Joseph Proudhon adopted anarchy and anarchist in his 1840 treatise What Is Property? to refer to anarchism, a new political philosophy and social movement that advocates stateless societies based on free and voluntary associations. Anarchists seek a system based on the abolition of all coercive hierarchy, in particular the state, and many advocate for the creation of a system of direct democracy, worker cooperatives or privatization. In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions.

In [15]:
predict_and_time(very_long_article, lg_nlp)

01/08/2023 19:59:22 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 30.89ba/s]
01/08/2023 19:59:24 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.12it/s]

Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). While Somalia's formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. In the case of the Transitional National Government and its successor the Transitional Federal Government, new interim judicial structures were formed through various international conferences. Despite some significant political differences between them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of pr

## Excerpt Tests (`lingmess`)

In [16]:
predict_and_time(short_excerpt, ling_nlp)

01/08/2023 19:59:24 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?ba/s]
01/08/2023 19:59:24 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.83s/it]

In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions. These institutions or free associations are generally modeled on nature.
Cluster 0
anarchy  - [20, 27]
It  - [123, 125]
Cluster 1
voluntary institutions. - [321, 343]
These institutions or free associations  - [345, 384]
Elapsed Time: 3938.4689331054688ms


In [17]:
predict_and_time(longer_article, ling_nlp)

01/08/2023 19:59:28 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.01ba/s]
01/08/2023 19:59:28 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.87s/it]

Anarchy is a society without a government. It may also refer to a society or group of people that entirely rejects a set hierarchy. Anarchy was first used in English in 1539, meaning "an absence of government". Pierre-Joseph Proudhon adopted anarchy and anarchist in his 1840 treatise What Is Property? to refer to anarchism, a new political philosophy and social movement that advocates stateless societies based on free and voluntary associations. Anarchists seek a system based on the abolition of all coercive hierarchy, in particular the state, and many advocate for the creation of a system of direct democracy, worker cooperatives or privatization. In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation or any inhabited place that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions.

In [18]:
predict_and_time(very_long_article, ling_nlp)

01/08/2023 19:59:32 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 62.46ba/s]
01/08/2023 19:59:32 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.87s/it]

Following the outbreak of the civil war in Somalia and the ensuing collapse of the central government, residents reverted to local forms of conflict resolution, either secular, traditional or Islamic law, with a provision for appeal of all sentences. The legal structure in the country was divided along three lines, namely civil law, religious law and customary law (xeer). While Somalia's formal judicial system was largely destroyed after the fall of the Siad Barre regime, it was later gradually rebuilt and administered under different regional governments such as the autonomous Puntland and Somaliland macro-regions. In the case of the Transitional National Government and its successor the Transitional Federal Government, new interim judicial structures were formed through various international conferences. Despite some significant political differences between them, all of these administrations shared similar legal structures, much of which were predicated on the judicial systems of pr

## Without using Spacy

In [19]:
model = FCoref()

01/08/2023 19:59:38 - INFO - 	 missing_keys: []
01/08/2023 19:59:38 - INFO - 	 unexpected_keys: []
01/08/2023 19:59:38 - INFO - 	 mismatched_keys: []
01/08/2023 19:59:38 - INFO - 	 error_msgs: []
01/08/2023 19:59:38 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


In [20]:
COLORS = ['green', 'red', 'yellow', 'blue', 'magenta', 'cyan']

def get_color(index):
    return COLORS[index % len(COLORS)]

def cprint(text, color, newline=True):
    print(colored(text, color), end='\n' if newline else ' ')
    
def render_clusters(prediction, text):
    corefs = prediction.get_clusters(as_strings=False)
    for i, cluster in enumerate(corefs):
        cprint(f'Cluster {i}', get_color(i))
        for phrase_start, phrase_end in cluster:
            cprint(f'{text[phrase_start:phrase_end + 1]} - [{phrase_start}, {phrase_end}]', get_color(i))
            
def predict_and_time(document, pipeline):
    start = time.time()
    prediction = pipeline.predict(document)
    end = time.time()
    print(document)
    render_clusters(prediction, document)
    print(f"Elapsed Time: {(end - start) * 1000}ms");

In [21]:
angelfish_article = "The queen angelfish (Holacanthus ciliaris), also known as the blue angelfish, "\
"golden angelfish, or yellow angelfish, is a species of marine angelfish found in the western Atlantic Ocean. "\
"It is a benthic (ocean floor) warm-water species that lives in coral reefs. "\
"It is recognized by its blue and yellow coloration and a distinctive spot or 'crown' on its forehead. "\
"This crown distinguishes it from the closely related and similar-looking Bermuda "\
"blue angelfish (Holacanthus bermudensis), with which it overlaps in range and can interbreed. "\
"Adult queen angelfish are selective feeders and primarily eat sponges. Their social structure consists "\
"of harems which include one male and up to four females. "\
"They live within a territory where the females forage separately and are tended to by the male. "\
"Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. "\
"Juveniles of the species have different coloration than adults and act as cleaner fish."\
"Adult queen angelfish are selective feeders and primarily eat sponges. "\
"Their social structure consists of harems which include one male and up to four females. "\
"They live within a territory where the females forage separately and are tended to by the male. "\
"Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. "\
"Juveniles of the species have different coloration than adults and act as cleaner fish. "\
"The queen angelfish is popular in the aquarium trade and has been a particularly common exported "\
"species from Brazil. In 2010, the queen angelfish was assessed as least concern by the International "\
"Union for Conservation of Nature as the wild population appeared to be stable." 

In [22]:
predict_and_time(angelfish_article, model)

01/08/2023 19:59:38 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.75ba/s]
01/08/2023 19:59:38 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.06it/s]

The queen angelfish (Holacanthus ciliaris), also known as the blue angelfish, golden angelfish, or yellow angelfish, is a species of marine angelfish found in the western Atlantic Ocean. It is a benthic (ocean floor) warm-water species that lives in coral reefs. It is recognized by its blue and yellow coloration and a distinctive spot or 'crown' on its forehead. This crown distinguishes it from the closely related and similar-looking Bermuda blue angelfish (Holacanthus bermudensis), with which it overlaps in range and can interbreed. Adult queen angelfish are selective feeders and primarily eat sponges. Their social structure consists of harems which include one male and up to four females. They live within a territory where the females forage separately and are tended to by the male. Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. Juveniles of the species have different coloration than adults and act as cleaner fish.Adult quee

## Full Article Test

In [24]:
angelfish_article = "The queen angelfish (Holacanthus ciliaris), also known as the blue angelfish, "\
"golden angelfish, or yellow angelfish, is a species of marine angelfish found in the western Atlantic Ocean. "\
"It is a benthic (ocean floor) warm-water species that lives in coral reefs. "\
"It is recognized by its blue and yellow coloration and a distinctive spot or 'crown' on its forehead. "\
"This crown distinguishes it from the closely related and similar-looking Bermuda "\
"blue angelfish (Holacanthus bermudensis), with which it overlaps in range and can interbreed. "\
"Adult queen angelfish are selective feeders and primarily eat sponges. Their social structure consists "\
"of harems which include one male and up to four females. "\
"They live within a territory where the females forage separately and are tended to by the male. "\
"Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. "\
"Juveniles of the species have different coloration than adults and act as cleaner fish."\
"Adult queen angelfish are selective feeders and primarily eat sponges. "\
"Their social structure consists of harems which include one male and up to four females. "\
"They live within a territory where the females forage separately and are tended to by the male. "\
"Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. "\
"Juveniles of the species have different coloration than adults and act as cleaner fish. "\
"The queen angelfish is popular in the aquarium trade and has been a particularly common exported "\
"species from Brazil. In 2010, the queen angelfish was assessed as least concern by the International "\
"Union for Conservation of Nature as the wild population appeared to be stable. " \
"The queen angelfish was first described as Chaetodon ciliaris in 1758 by Carl Linnaeus in the 10th edition "\
"of his Systema Naturae, with the type locality given as the 'Western Atlantic/Caribbean'. "\
"In 1802 it was moved by French naturalist Bernard Germain de Lacépède to the genus Holacanthus, "\
"the name of which is derived from the Ancient Greek words 'holos' (full) and 'akantha' (thorn). "\
"Its specific name ciliaris means 'fringed', a reference to its squamis ciliatis ('ciliate scales'). "\
"Other common names for the species include 'blue angelfish', 'golden angelfish' and 'yellow angelfish'. "\
"Marine angelfish of the genus Holacanthus likely emerged between 10.2 and 7.6 million years ago (mya). "\
"The most basal species is the Guinean angelfish (Holacanthus africanus) off the coast of West Africa, "\
"indicating that the lineage colonized the Atlantic from the Indian Ocean. "\
"The closure of the Isthmus of Panama 3.5–3.1 mya led to the splitting off of the Tropical Eastern "\
"Pacific species. The closest relative and sister species of the queen angelfish is the sympatric "\
"and similar Bermuda blue angelfish (H. bermudensis), from which it split around 1.5 mya. "\
"They are known to interbreed, producing a hybrid known as the Townsend angelfish which has "\
"features similar to both parent species. The Townsend angelfish is fertile, and individuals "\
"can breed both with each other and with the two parent species."

In [25]:
predict_and_time(angelfish_article, model)

01/08/2023 20:05:07 - INFO - 	 Tokenize 1 inputs...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 32.00ba/s]
01/08/2023 20:05:07 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.41it/s]

The queen angelfish (Holacanthus ciliaris), also known as the blue angelfish, golden angelfish, or yellow angelfish, is a species of marine angelfish found in the western Atlantic Ocean. It is a benthic (ocean floor) warm-water species that lives in coral reefs. It is recognized by its blue and yellow coloration and a distinctive spot or 'crown' on its forehead. This crown distinguishes it from the closely related and similar-looking Bermuda blue angelfish (Holacanthus bermudensis), with which it overlaps in range and can interbreed. Adult queen angelfish are selective feeders and primarily eat sponges. Their social structure consists of harems which include one male and up to four females. They live within a territory where the females forage separately and are tended to by the male. Breeding in the species occurs near a full moon. The transparent eggs float in the water until they hatch. Juveniles of the species have different coloration than adults and act as cleaner fish.Adult quee

## Conclusion

The lingmess model is definitely more accurate, but too slow. 

The speed is very inconsistent.